<a href="https://colab.research.google.com/github/DaheeKo/Deep-learning-study/blob/main/Chap_10_Vanila_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Numpy로 구현해보기**

< 방법 >

hidden_state_t 초기 은닉 상태를 0벡터로 초기화

각 시점마다 input_t을 받아서

output_t: 입력과 은닉상태를 가지고 연산(tanh)

계산결과 -> 출력이면서, 다시 현재 시점의 은닉상태가 됨

In [2]:
import numpy as np

timesteps = 10 # 시점의 수, NLP에서 보통 문장의 길이
input_size = 4 # 입력의 차원, NLP에서 보통 단어 벡터의 차원 
hidden_size = 8 # 은닉 상태의 크기, 메모리 셀의 용량

inputs = np.random.random((timesteps, input_size)) # 입력에 해당, 2D 텐서
hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태 초기화

Wx = np.random.random((hidden_size, input_size)) # 입력에 대한 가중치
Wh = np.random.random((hidden_size, hidden_size)) # 은닉상태에 대한 가중치
b = np.random.random((hidden_size,)) # 편향
 # => 출력은 (hidden_size,)로 나옴

total_hidden_state = []

for input_t in inputs:
  output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)
  total_hidden_state.append(output_t)

**2. 파이토치의 nn.RNN()로 구현**

In [3]:
import torch
import torch.nn as nn

 # 하이퍼파라미터 정의
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

 # 입력 텐서 정의
inputs = torch.Tensor(1, 10, 5)  # (batch_size, time_steps, input_size)

 # RNN 셀 만들기
  # RNN 셀은 두 개의 입력 리턴: (모든 시점의 은닉 상태들, 마지막 시점의 은닉 상태)
cell = nn.RNN(input_size, hidden_size, batch_first=True)
outputs, _status = cell(inputs) # 모든 시점의 은닉 상태들, 마지막 시점의 은닉 상태 
print(outputs.shape) # : torch.Size([1, 10, 8]) = 10번의 시점 동안 8차원의 은닉상태가 출력
print(_status.shape) # : torch.Size([1, 1, 8]) = 마지막 시점의 은닉상태 크기

print('---------------------------')
# 깊은 순환 신경망
cell_D = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True) # 은닉층이 2개이도록 num_layers = 2
outputs, _status = cell_D(inputs)
print(outputs.shape) # : torch.Size([1, 10, 8]) = 마지막 층의 모든 시점의 은닉상태라서 동일 
print(_status.shape) # : torch.Size([2, 1, 8]) = 마지막 시점의 은닉상태 (층의 개수, 배치 크기, 은닉상태의 크기)

print('---------------------------')
# 양방향 순환 신경망
cell_B = cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional = True) # 은닉층 2개 & 양방향(bidirectional = True)
outputs, _status = cell_B(inputs)
print(outputs.shape) # : torch.Size([1, 10, 16]) = (배치크기, 시퀀스 길이, 은닉상태 크기x2) <= 양방향의 은닉상태가 연결돼서
print(_status.shape) # : torch.Size([4, 1, 8]) = (층 개수x2, 배치크기, 은닉상태 크기x2) : 정방향 기준으로는 마지막 시점에 해당되면서, 역방향 기준에서는 첫번째 시점에 해당되는 시점의 출력값을 층의 개수만큼 쌓아 올린 결과값 ??

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])
---------------------------
torch.Size([1, 10, 8])
torch.Size([2, 1, 8])
---------------------------
torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


**10장 chap2 장단기 메모리 LSTM**

바닐라 RNN : 시점(time step)이 길어질 수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생 - 장기 의존성 문제

Long Short-Term Memory (LSTM)

: 은닉층의 메모리 셀에 입력 게이트, 삭제 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정함

- 입력게이트 (i_t, g_t -> 셀 상태에서 기억할 양 정함) : 현재 시점의 입력을 얼마나 반영할 지 결정

    => i_t: 입력&이전 은닉상태 -> 시그모이드

    => g_t: 입력&이전 은닉상태 -> tanh
- 삭제게이트 (f_t: 삭제과정 거친 후의 정보의 양) : 이전 시점의 입력을 얼마나 반영할 지를 의미

    => f_t: 입력&이전 은닉상태 -> 시그모이드 

- 셀 상태(장기상태) (C_t)

    => C_t = f_t * C_t-1 + i_t * g_t ( *는 원소별 곱), i_t * g_t는 이번에 선택된 기억할 값

- 출력게이트(o_t)와 은닉상태(단기상태) (h_t)

    => o_t: 입력&이전 은닉상태 -> 시그모이드

    => h_t = o_t * tanh(C_t) : 현재의 은닉상태 & 출력층으로 향함

In [4]:
# nn.RNN(input_dim, hidden_size, batch_fisrt=True) # 이거를
# nn.LSTM(input_dim, hidden_size, batch_fisrt=True) # 이렇게 바꿔주면 됨